# Load Data
## EISCAT Data

In [1]:
import h5py
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta
import glob
import sys

%load_ext autoreload
%autoreload 2

pd.set_option('display.max_columns', None)

# Get the absolute path of the project root (fpi_ml)
project_root = "/Users/elliotdable/Documents/PhD/research/fpi_ml"
sys.path.append(project_root)

# Now try importing using 'src.'
from src.load_data import eiscat_hdf5_loader

In [2]:
import numpy as np
import xarray as xr

# Now try importing using 'src.'
from src.load_data import eiscat_netCDF_loader, load_full_eiscat_netCDF_data

# Directory containing the files
directory = "/Users/elliotdable/Documents/PhD/research/fpi_ml/data/eiscat_data"

eiscat_df = load_full_eiscat_netCDF_data(directory)

Attempting to load the file as an xarray.Dataset (e.g., if it is in NetCDF format)...
File successfully loaded as an xarray.Dataset.
Attempting to load the file as an xarray.Dataset (e.g., if it is in NetCDF format)...
File successfully loaded as an xarray.Dataset.
Attempting to load the file as an xarray.Dataset (e.g., if it is in NetCDF format)...
File successfully loaded as an xarray.Dataset.
Attempting to load the file as an xarray.Dataset (e.g., if it is in NetCDF format)...
File successfully loaded as an xarray.Dataset.
Attempting to load the file as an xarray.Dataset (e.g., if it is in NetCDF format)...
File successfully loaded as an xarray.Dataset.
Attempting to load the file as an xarray.Dataset (e.g., if it is in NetCDF format)...
File successfully loaded as an xarray.Dataset.
Attempting to load the file as an xarray.Dataset (e.g., if it is in NetCDF format)...
File successfully loaded as an xarray.Dataset.
Attempting to load the file as an xarray.Dataset (e.g., if it is in N

In [3]:
eiscat_df['unix_time'] = (eiscat_df['time'] - pd.Timestamp("1970-01-01")) // pd.Timedelta("1s")
eiscat_df = eiscat_df.dropna()
eiscat_df = eiscat_df.round(6).drop_duplicates()
eiscat_df.head()

,altitude,time,Ne,Te,Ti,Vi,Ne_err,Te_err,Ti_err,Vi_err,azimuth,elevation,observation_latitude,observation_longitude,unix_time
549,50,2014-06-26 09:05:04.824221500,1.411483e+08,283.339270,283.339270,-3.395707,5.018694e+07,0.0,0.0,0.0,183.964106,89.975813,69.138307,19.138174,1403773504
550,50,2014-06-26 09:15:00.024046000,8.283085e+07,283.339190,283.339190,-3.395707,6.579822e+07,0.0,0.0,0.0,183.960000,89.974880,69.138305,19.138267,1403774100
551,50,2014-06-26 09:25:00.024176000,1.606274e+08,283.339332,283.339332,-3.395707,5.146058e+07,0.0,0.0,0.0,183.960000,89.970000,69.138305,19.138267,1403774700
552,50,2014-06-26 09:35:00.024412000,1.034546e+08,283.440070,283.440070,-3.395707,5.995907e+07,0.0,0.0,0.0,183.967975,89.970000,69.138309,19.138088,1403775300
553,50,2014-06-26 09:45:00.024205500,3.441295e+08,283.339672,283.339672,-3.395707,3.119477e+07,0.0,0.0,0.0,183.970403,89.970565,69.138311,19.138033,1403775900


In [4]:
#eiscat_filtered = eiscat_df[eiscat_df['unix_time']> 1600000000]
eiscat_filtered = eiscat_df[eiscat_df['altitude'] > 100]
eiscat_filtered = eiscat_filtered[eiscat_filtered['altitude'] < 300]
eiscat_filtered = eiscat_filtered[eiscat_filtered['observation_latitude'] > 66]
eiscat_filtered = eiscat_filtered[eiscat_filtered['observation_latitude'] < 70]
eiscat_filtered = eiscat_filtered[eiscat_filtered['observation_longitude'] > 15]
eiscat_filtered = eiscat_filtered[eiscat_filtered['observation_longitude'] < 27]
eiscat_filtered = eiscat_filtered.rename(columns={
    'time': 'datetime', 
    'Ne': 'electron_density',
    'Te': 'electron_temperature',
    'Ti': 'ion_temperature',
    'Vi': 'ion_velocity',
    'Ne_err': 'electron_density_error',
    'Te_err': 'electron_temperature_error',
    'Ti_err': 'ion_temperature_error',
    'Vi_err': 'ion_velocity_error'
}) 

eiscat_filtered = eiscat_filtered[['datetime', 'unix_time', 'altitude', 'azimuth', 'elevation', 'observation_latitude',
       'observation_longitude', 'electron_density', 'electron_temperature',
       'ion_temperature', 'ion_velocity', 'electron_density_error',
       'electron_temperature_error', 'ion_temperature_error',
       'ion_velocity_error']]
       
eiscat_df = eiscat_filtered.copy()

del eiscat_filtered

In [5]:
eiscat_df.columns

Index(['datetime', 'unix_time', 'altitude', 'azimuth', 'elevation',
       'observation_latitude', 'observation_longitude', 'electron_density',
       'electron_temperature', 'ion_temperature', 'ion_velocity',
       'electron_density_error', 'electron_temperature_error',
       'ion_temperature_error', 'ion_velocity_error'],
      dtype='object')

## FPI Data

In [6]:
fpi_df = pd.read_csv('/Users/elliotdable/Documents/PhD/research/fpi_ml/data/fpi_data/indra/indra_fpi_data.csv')

fpi_df['datetime'] = pd.to_datetime(fpi_df['datetime'])
fpi_df['unix_time'] = (fpi_df['datetime'] - pd.Timestamp("1970-01-01")) // pd.Timedelta("1s")
fpi_df = fpi_df.dropna()
fpi_df = fpi_df.round(6).drop_duplicates()

fpi_df = fpi_df[['datetime', 'unix_time', 'site', 'mirror', 'look_direction', 'azimuth_angle',
       'altitude', 'fpi_latitude', 'fpi_longitude', 'observation_latitude',
       'observation_longitude', 'intensity', 'intensity_error', 'wind_speed',
       'wind_speed_error', 'wind_direction', 'temperature',
       'temperature_error', 'chi_squared', 'snr']]

In [7]:
fpi_df

,datetime,unix_time,site,mirror,look_direction,azimuth_angle,altitude,fpi_latitude,fpi_longitude,observation_latitude,observation_longitude,intensity,intensity_error,wind_speed,wind_speed_error,wind_direction,temperature,temperature_error,chi_squared,snr
2,2013-09-11 18:54:39,1378925679,kiruna_red,4,W,270.0,240.0,67.87,21.03,67.870000,12.936098,1084.94,7.11,1470.32,5.52,90.0,699.96,11.48,2.00,18045.0
3,2013-09-11 18:55:36,1378925736,kiruna_red,1,N,0.0,240.0,67.87,21.03,70.919049,21.030000,943.89,6.64,1545.55,5.88,180.0,700.00,12.31,2.18,16207.0
4,2013-09-11 18:56:29,1378925789,kiruna_red,11,Kir A,44.0,240.0,67.87,21.03,70.063302,26.652497,881.21,5.18,1547.72,4.94,224.0,696.16,10.24,3.42,23563.0
5,2013-09-11 18:57:21,1378925841,kiruna_red,2,E,90.0,240.0,67.87,21.03,67.870000,29.123902,810.09,6.17,1572.24,4.48,270.0,524.12,9.83,2.57,14737.0
8,2013-09-11 19:00:09,1378926009,kiruna_red,12,Kir B,157.0,240.0,67.87,21.03,65.063336,24.192540,562.40,4.32,1546.35,6.48,337.0,698.16,13.41,1.46,13173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3062585,2025-02-27 04:56:14,1740632174,kiruna_green,4,W,270.0,120.0,67.87,21.03,67.870000,16.983049,1239.55,3.10,174.53,1.80,90.0,842.25,5.37,0.28,92600.0
3062586,2025-02-27 04:56:40,1740632200,kiruna_green,5,NW,315.0,120.0,67.87,21.03,68.948001,18.168373,1690.95,3.20,155.05,1.32,135.0,785.29,3.76,0.58,160094.0
3062588,2025-02-27 05:02:28,1740632548,kiruna_green,1,N,0.0,120.0,67.87,21.03,69.394524,21.030000,1148.16,3.93,123.43,2.44,180.0,822.67,7.16,0.22,49209.0
3062589,2025-02-27 05:04:01,1740632641,kiruna_green,4,W,270.0,120.0,67.87,21.03,67.870000,16.983049,701.73,3.23,154.25,3.29,90.0,826.18,9.67,0.33,27419.0


## IMF Data

In [8]:
# Now try importing using 'src.'
from src.load_data import load_omni_imf_data

imf_dir = '/Users/elliotdable/Documents/PhD/research/fpi_ml/data/imf_data'
imf_df = pd.read_csv('/Users/elliotdable/Documents/PhD/research/fpi_ml/data/imf_data/omni_imf_data_2010_2025.csv')
#imf_df = load_omni_imf_data(imf_dir)
imf_df.columns

Index(['datetime', 'day', 'hour', 'minute', 'bx_gse_gsm', 'by_gse', 'bz_gse',
       'b_vector_rms_sd', 'vx_kms', 'vy_kms', 'vz_kms', 'proton_density',
       'proton_temperature', 'al_index', 'au_index'],
      dtype='object')

In [9]:
'''# Step 1: Select numeric columns only
numeric_df = imf_df.select_dtypes(include=[np.number])

# Step 2: Create a mask of rows where all values are <= 99999
mask = (numeric_df <= 99999).all(axis=1)

# Step 3: Apply the mask to the full DataFrame
df_cleaned = imf_df[mask].reset_index(drop=True)

df_cleaned.to_csv('/Users/elliotdable/Documents/PhD/research/fpi_ml/data/imf_data/omni_imf_data_2010_2025.csv', index=False)'''

"# Step 1: Select numeric columns only\nnumeric_df = imf_df.select_dtypes(include=[np.number])\n\n# Step 2: Create a mask of rows where all values are <= 99999\nmask = (numeric_df <= 99999).all(axis=1)\n\n# Step 3: Apply the mask to the full DataFrame\ndf_cleaned = imf_df[mask].reset_index(drop=True)\n\ndf_cleaned.to_csv('/Users/elliotdable/Documents/PhD/research/fpi_ml/data/imf_data/omni_imf_data_2010_2025.csv', index=False)"

In [11]:
import pandas as pd
from datetime import timedelta

# Define column names from the header
columns = [
    'YYYY', 'MM', 'DD', 'days', 'days_m', 'Bsr', 'dB',
    'Kp1', 'Kp2', 'Kp3', 'Kp4', 'Kp5', 'Kp6', 'Kp7', 'Kp8',
    'ap1', 'ap2', 'ap3', 'ap4', 'ap5', 'ap6', 'ap7', 'ap8',
    'Ap', 'SN', 'F10_7obs', 'F10_7adj', 'D'
]

# Read the data, skipping the comment line
df = pd.read_csv(
    '/Users/elliotdable/Documents/PhD/research/fpi_ml/data/geomag_data/Kp_ap_Ap_SN_F107_since_1932.txt',
    sep='\s+',
    comment='#',
    names=columns
)

# Create a 'date' column
df['date'] = pd.to_datetime(df[['YYYY', 'MM', 'DD']].rename(columns={'YYYY': 'year', 'MM': 'month', 'DD': 'day'}))

# Melt Kp columns into long format
kp_cols = [f'Kp{i}' for i in range(1, 9)]
kp_df = df[['date'] + kp_cols].melt(id_vars='date', var_name='Kp_index', value_name='Kp')
kp_df['hour'] = (kp_df['Kp_index'].str.extract('(\d)').astype(int) - 1) * 3
kp_df['datetime'] = kp_df['date'] + pd.to_timedelta(kp_df['hour'], unit='h')

# Melt ap columns into long format
ap_cols = [f'ap{i}' for i in range(1, 9)]
ap_df = df[['date'] + ap_cols].melt(id_vars='date', var_name='ap_index', value_name='ap')
ap_df['hour'] = (ap_df['ap_index'].str.extract('(\d)').astype(int) - 1) * 3
ap_df['datetime'] = ap_df['date'] + pd.to_timedelta(ap_df['hour'], unit='h')

# Merge Kp and ap by datetime
geomag_df = pd.merge(kp_df[['datetime', 'date', 'Kp']], ap_df[['datetime', 'ap']], on='datetime')

# Merge daily values onto every row
daily_vals = df[['date', 'Ap', 'SN', 'F10_7obs', 'F10_7adj']]
geomag_df = geomag_df.merge(daily_vals, on='date', how='left')

# Sort and reset index
geomag_df = geomag_df.sort_values('datetime').reset_index(drop=True)
geomag_df = geomag_df.drop(columns=['date'])
geomag_df = geomag_df[geomag_df['datetime'] > '2010-01-01 00:00:00']
geomag_df = geomag_df[['datetime', 'ap', 'SN', 'F10_7adj']]
geomag_df

<>:15: SyntaxWarning: invalid escape sequence '\s'
<>:26: SyntaxWarning: invalid escape sequence '\d'
<>:32: SyntaxWarning: invalid escape sequence '\d'
<>:15: SyntaxWarning: invalid escape sequence '\s'
<>:26: SyntaxWarning: invalid escape sequence '\d'
<>:32: SyntaxWarning: invalid escape sequence '\d'
/var/folders/pf/x6mn9jt96z14ybg9rxsvmb100000gn/T/ipykernel_66042/2789833431.py:15: SyntaxWarning: invalid escape sequence '\s'
  sep='\s+',
/var/folders/pf/x6mn9jt96z14ybg9rxsvmb100000gn/T/ipykernel_66042/2789833431.py:26: SyntaxWarning: invalid escape sequence '\d'
  kp_df['hour'] = (kp_df['Kp_index'].str.extract('(\d)').astype(int) - 1) * 3
/var/folders/pf/x6mn9jt96z14ybg9rxsvmb100000gn/T/ipykernel_66042/2789833431.py:32: SyntaxWarning: invalid escape sequence '\d'
  ap_df['hour'] = (ap_df['ap_index'].str.extract('(\d)').astype(int) - 1) * 3


,datetime,ap,SN,F10_7adj
227921,2010-01-01 03:00:00,0,18,72.7
227922,2010-01-01 06:00:00,0,18,72.7
227923,2010-01-01 09:00:00,0,18,72.7
227924,2010-01-01 12:00:00,0,18,72.7
227925,2010-01-01 15:00:00,0,18,72.7
...,...,...,...,...
272795,2025-05-11 09:00:00,6,72,128.3
272796,2025-05-11 12:00:00,15,72,128.3
272797,2025-05-11 15:00:00,5,72,128.3
272798,2025-05-11 18:00:00,32,72,128.3


## Joining the IMF to the EISCAT and FPI data

In [12]:
imf_df.columns, geomag_df.columns, fpi_df.columns

(Index(['datetime', 'day', 'hour', 'minute', 'bx_gse_gsm', 'by_gse', 'bz_gse',
        'b_vector_rms_sd', 'vx_kms', 'vy_kms', 'vz_kms', 'proton_density',
        'proton_temperature', 'al_index', 'au_index'],
       dtype='object'),
 Index(['datetime', 'ap', 'SN', 'F10_7adj'], dtype='object'),
 Index(['datetime', 'unix_time', 'site', 'mirror', 'look_direction',
        'azimuth_angle', 'altitude', 'fpi_latitude', 'fpi_longitude',
        'observation_latitude', 'observation_longitude', 'intensity',
        'intensity_error', 'wind_speed', 'wind_speed_error', 'wind_direction',
        'temperature', 'temperature_error', 'chi_squared', 'snr'],
       dtype='object'))

In [13]:
import pandas as pd

def join_imf_to_param_df(param_df, imf_df, geomag_df):
    # Ensure datetime columns are in datetime format
    param_df = param_df.copy()
    imf_df = imf_df.copy()
    geomag_df = geomag_df.copy()

    param_df['datetime'] = pd.to_datetime(param_df['datetime'])
    imf_df['datetime'] = pd.to_datetime(imf_df['datetime'])
    geomag_df['datetime'] = pd.to_datetime(geomag_df['datetime'])
    # Sort both dataframes by datetime for merge_asof
    param_df = param_df.sort_values('datetime')
    imf_df = imf_df.sort_values('datetime')
    geomag_df = geomag_df.sort_values('datetime')
    
    # Perform the merge_asof (nearest match backward by default)
    fpi_imf_df = pd.merge_asof(
        param_df, imf_df, on='datetime', direction='nearest'
    )
    fpi_imf_df = pd.merge_asof(
        fpi_imf_df, geomag_df, on='datetime', direction='nearest'
    )

    fpi_imf_df = fpi_imf_df.drop(columns=['day', 'hour', 'minute'])
    
    return fpi_imf_df
    
fpi_imf_df = join_imf_to_param_df(fpi_df, imf_df, geomag_df)
eiscat_imf_df = join_imf_to_param_df(eiscat_df, imf_df, geomag_df)

In [14]:
eiscat_imf_df.columns, fpi_imf_df.columns

(Index(['datetime', 'unix_time', 'altitude', 'azimuth', 'elevation',
        'observation_latitude', 'observation_longitude', 'electron_density',
        'electron_temperature', 'ion_temperature', 'ion_velocity',
        'electron_density_error', 'electron_temperature_error',
        'ion_temperature_error', 'ion_velocity_error', 'bx_gse_gsm', 'by_gse',
        'bz_gse', 'b_vector_rms_sd', 'vx_kms', 'vy_kms', 'vz_kms',
        'proton_density', 'proton_temperature', 'al_index', 'au_index', 'ap',
        'SN', 'F10_7adj'],
       dtype='object'),
 Index(['datetime', 'unix_time', 'site', 'mirror', 'look_direction',
        'azimuth_angle', 'altitude', 'fpi_latitude', 'fpi_longitude',
        'observation_latitude', 'observation_longitude', 'intensity',
        'intensity_error', 'wind_speed', 'wind_speed_error', 'wind_direction',
        'temperature', 'temperature_error', 'chi_squared', 'snr', 'bx_gse_gsm',
        'by_gse', 'bz_gse', 'b_vector_rms_sd', 'vx_kms', 'vy_kms', 'vz_kms',
   

# Training GNN

## Setting Up The Network (Nodes and Edges)

In [15]:
eiscat_imf_df.shape, fpi_imf_df.shape

((2384630, 29), (2056266, 34))

In [21]:
fpi_2024 = fpi_imf_df[(fpi_imf_df['datetime'] > '2013-01-01 00:00:00') & (fpi_imf_df['datetime'] < '2025-01-01 00:00:00')]

In [19]:
fpi_2024.to_csv('/Users/elliotdable/Documents/PhD/research/fpi_ml/data/fpi_data/indra/scandinavia_2013_2024.csv', index=False)
#geomag_df.to_csv('/Users/elliotdable/Documents/PhD/research/fpi_ml/data/geomag_data/geomag_data.csv', index=False)

In [22]:
fpi_2024

,datetime,unix_time,site,mirror,look_direction,azimuth_angle,altitude,fpi_latitude,fpi_longitude,observation_latitude,observation_longitude,intensity,intensity_error,wind_speed,wind_speed_error,wind_direction,temperature,temperature_error,chi_squared,snr,bx_gse_gsm,by_gse,bz_gse,b_vector_rms_sd,vx_kms,vy_kms,vz_kms,proton_density,proton_temperature,al_index,au_index,ap,SN,F10_7adj
0,2013-09-11 18:54:39,1378925679,kiruna_red,4,W,270.0,240.0,67.87,21.03,67.870000,12.936098,1084.94,7.11,1470.32,5.52,90.0,699.96,11.48,2.00,18045.0,2.97,-0.90,0.65,2.33,-384.5,-21.3,-14.9,3.72,57533.0,-107,73,6,60,94.1
1,2013-09-11 18:55:36,1378925736,kiruna_red,1,N,0.0,240.0,67.87,21.03,70.919049,21.030000,943.89,6.64,1545.55,5.88,180.0,700.00,12.31,2.18,16207.0,2.73,-0.74,1.53,1.96,-384.8,-9.1,-8.2,3.34,71777.0,-93,71,6,60,94.1
2,2013-09-11 18:56:29,1378925789,kiruna_red,11,Kir A,44.0,240.0,67.87,21.03,70.063302,26.652497,881.21,5.18,1547.72,4.94,224.0,696.16,10.24,3.42,23563.0,2.73,-0.74,1.53,1.96,-384.8,-9.1,-8.2,3.34,71777.0,-93,71,6,60,94.1
3,2013-09-11 18:57:21,1378925841,kiruna_red,2,E,90.0,240.0,67.87,21.03,67.870000,29.123902,810.09,6.17,1572.24,4.48,270.0,524.12,9.83,2.57,14737.0,2.71,-1.69,1.18,2.20,-383.5,-7.6,-8.3,3.60,70493.0,-84,69,6,60,94.1
4,2013-09-11 19:00:09,1378926009,kiruna_red,12,Kir B,157.0,240.0,67.87,21.03,65.063336,24.192540,562.40,4.32,1546.35,6.48,337.0,698.16,13.41,1.46,13173.0,2.70,-3.19,-0.51,0.46,-380.0,-25.8,-5.4,3.90,58949.0,-73,72,6,60,94.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1892276,2024-12-31 23:59:09,1735689549,kiruna_red,12,Kir B,157.0,240.0,67.87,21.03,65.063336,24.192540,9056.80,14.76,250.05,1.45,157.0,1522.61,6.96,0.18,244204.0,-6.01,12.49,-4.57,1.94,-427.1,-29.8,50.9,25.48,86657.0,-352,188,27,201,211.9
1892277,2024-12-31 23:59:22,1735689562,kiruna_green,4,W,270.0,120.0,67.87,21.03,67.870000,16.983049,722.14,1.69,195.89,2.06,90.0,1376.44,8.69,0.34,114364.0,-6.01,12.49,-4.57,1.94,-427.1,-29.8,50.9,25.48,86657.0,-352,188,27,201,211.9
1892278,2024-12-31 23:59:35,1735689575,kiruna_red,3,S,180.0,240.0,67.87,21.03,64.820951,21.030000,8164.25,13.40,209.44,1.45,180.0,1509.93,6.95,0.24,240324.0,-6.01,12.49,-4.57,1.94,-427.1,-29.8,50.9,25.48,86657.0,-352,188,27,201,211.9
1892279,2024-12-31 23:59:48,1735689588,kiruna_green,5,NW,315.0,120.0,67.87,21.03,68.948001,18.168373,793.38,1.73,192.85,1.94,135.0,1403.02,8.30,0.33,131270.0,-6.01,12.49,-4.57,1.94,-427.1,-29.8,50.9,25.48,86657.0,-352,188,27,201,211.9
